# Assignment 5 - Text Analysis
An explanation this assignment could be found in the .pdf explanation document

> # Presented by:
Haim Atiya<br />
Ido Peer


## Materials to review for this assignment
<h4>From Moodle:</h4> 
<h5><u>Review the notebooks regarding the following python topics</u>:</h5>
<div class="alert alert-info">
&#x2714; <b>Working with strings</b> (tutorial notebook)<br/>
&#x2714; <b>Text Analysis</b> (tutorial notebook)<br/>
&#x2714; <b>Hebrew text analysis tools (tokenizer, wordnet)</b> (moodle example)<br/>
&#x2714; <b>(brief review) All previous notebooks</b><br/>
</div> 
<h5><u>Review the presentations regarding the following topics</u>:</h5>
<div class="alert alert-info">
&#x2714; <b>Text Analysis</b> (lecture presentation)<br/>
&#x2714; <b>(brief review) All other presentations</b><br/>
</div>

## Preceding Step - import modules (packages)
This step is necessary in order to use external modules (packages). <br/>

In [1]:
# --------------------------------------
import pandas as pd
import numpy as np
# --------------------------------------


# --------------------------------------
# ------------- visualizations:
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap
# --------------------------------------


# ---------------------------------------
import sklearn
from sklearn import preprocessing, metrics, pipeline, model_selection, feature_extraction 
from sklearn import naive_bayes, linear_model, svm, neural_network, neighbors, tree
from sklearn import decomposition, cluster

from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV 
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.metrics import precision_score, recall_score, f1_score
from sklearn.metrics import mean_squared_error, r2_score, silhouette_score
from sklearn.preprocessing import MinMaxScaler, StandardScaler, LabelEncoder

from sklearn.svm import LinearSVC
from sklearn.neural_network import MLPClassifier
from sklearn.linear_model import Perceptron, SGDClassifier
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
from sklearn.naive_bayes import MultinomialNB, GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
# ---------------------------------------

# select best features:
from sklearn.feature_selection import SelectKBest, chi2
#make scorer:
from sklearn.metrics import make_scorer

# Logistic Regression:
from sklearn.linear_model import LogisticRegression

# ----------------- output and visualizations: 
import warnings
from sklearn.exceptions import ConvergenceWarning
warnings.simplefilter("ignore")
warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.simplefilter("ignore", category=ConvergenceWarning)
# show several prints in one cell. This will allow us to condence every trick in one cell.
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
%matplotlib inline
pd.pandas.set_option('display.max_columns', None)
pd.set_option('display.float_format', lambda x: '%.3f' % x)
# ---------------------------------------

### Text analysis and String manipulation imports:

In [2]:
# --------------------------------------
# --------- Text analysis and Hebrew text analysis imports:
# vectorizers:
from sklearn.feature_extraction import text
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

# regular expressions:
import re
# --------------------------------------

### (optional) Hebrew text analysis - WordNet (for Hebrew)
Note: the WordNet is not a must

#### (optional) Only if you didn't install Wordnet (for Hebrew) use:

In [3]:
# word net installation:

# unmark if you want to use and need to install
# !pip install wn
# !python -m wn download omw-he:1.4

In [4]:
# word net import:

# unmark if you want to use:
# import wn

### (optional) Hebrew text analysis - hebrew_tokenizer (Tokenizer for Hebrew)
Note: the hebrew_tokenizer is not a must

#### (optional) Only if you didn't install hebrew_tokenizer use:

In [5]:
# Hebrew tokenizer installation:

# unmark if you want to use and need to install:
# !pip install hebrew_tokenizer

In [6]:
# Hebrew tokenizer import:

# unmark if you want to use:
#import hebrew_tokenizer as ht

### Reading input files
Reading input files for train annotated corpus (raw text data) corpus and for the test corpus

In [7]:
train_filename = 'annotated_corpus_for_train.csv'
test_filename  = 'corpus_for_test.csv'
df_train = pd.read_csv(train_filename, index_col=None, encoding='utf-8')
df_test  = pd.read_csv(test_filename, index_col=None, encoding='utf-8')

In [8]:
df_train.head(8)
df_train.shape
df_train.value_counts('gender')

,story,gender
0,"כשחבר הזמין אותי לחול, לא באמת חשבתי שזה יקרה,...",m
1,לפני שהתגייסתי לצבא עשיתי כל מני מיונים ליחידו...,m
2,מאז שהתחילו הלימודים חלומו של כל סטודנט זה הפנ...,f
3,"כשהייתי ילד, מטוסים היה הדבר שהכי ריתק אותי. ב...",m
4,‏הייתי מדריכה בכפר נוער ומתאם הכפר היינו צריכי...,f
5,לפני כ3 חודשים טסתי לרומא למשך שבוע. טסתי במטו...,f
6,אני כבר שנתיים נשוי והשנה אני ואישתי סוף סוף י...,m
7,השנה התחלנו שיפוץ בדירה שלנו בתל אביב. הדירה ה...,f


(753, 2)

gender
m    575
f    178
dtype: int64

In [9]:
df_test.head(3)
df_test.shape

,test_example_id,story
0,0,כל קיץ אני והמשפחה נוסעים לארצות הברית לוס אנג...
1,1,"הגעתי לשירות המדינה אחרי שנתיים כפעיל בתנועת ""..."
2,2,אחת האהבות הגדולות שלי אלו הכלבים שלי ושל אישת...


(323, 2)

### Your implementation:
Write your code solution in the following code-cells

# Preprocessing:

> ### Check for missing values:

In [10]:
# First of all, we will check if there are any missing values in the data set.
num_missing = df_train.isnull().sum().sort_values(ascending=False)
print("Num of missing values:\n%s" % num_missing)
# There are no missing values in the data set.


Num of missing values:
story     0
gender    0
dtype: int64


> ### Clear the text:

In [11]:
# Clean the text from punctuation marks, numbers, and other characters that are not letters.
def clear_text(text):
    # remove punctuation:
    text = re.sub(r'[^\w\s]', '', text)
    # remove numbers:
    text = re.sub(r'\d+', '', text)
    # remove special characters and symbols:
    text = re.sub(r'[^\w\s]', '', text)
    # remove multiple spaces:
    text = re.sub(r'\s+', ' ', text)
    return text

def clear_df(df):
    print ("Cleaning text...")
    df['story'] = df['story'].apply(clear_text)
    return df

> ### Vectorize the story:

In [12]:
# Now that we have cleaned the text, we will vectorize it using 2 methods:
# 1. CountVectorizer
# 2. TfidfVectorizer
# We will use the TfidfVectorizer as default, and we will compare the results of the two methods.
def vectorize(df,method='tfidf', test=False):
    print ("Vectorizing...")
    cpy = df.copy()
    if method == 'tfidf':
        vectorizer = TfidfVectorizer(ngram_range=(1, 1), min_df=5)
    elif method == 'count':
        vectorizer = CountVectorizer(ngram_range=(1, 1), min_df=5)

    vectorizer.fit_transform(cpy['story'])
    if test:
        vectorizer.transform(cpy['story'])
    
    cpy = pd.concat([cpy, pd.DataFrame(vectorizer.transform(cpy['story']).toarray())], axis=1)
    cpy.drop(['story'], axis=1, inplace=True)
    # remove blank rows:
    cpy = cpy.dropna()
    return cpy


> ### Feature selection:

In [13]:
# This function will select the best features from the vectorized data set in order to improve the model's performance.
# We will use the SelectKBest method, and we will compare the results of the two methods.
# this function inputs X and y and returns a dataframe with the best 80 features
def select_features(X, y):
    print ("Selecting features...")
    cpy = X.copy()
    selector = SelectKBest(chi2, k=1300)
    selector.fit_transform(cpy, y)
    cpy = pd.DataFrame(selector.transform(cpy))
    return cpy

> ### Scale the data:

In [14]:
# We are going to scale the data using 2 methods:
# 1. MinMaxScaler
# 2. StandardScaler
# We will use the MinMax as default, and we will compare the results of the two methods.
def scale(df, method='minmax'):
    print ("Scaling...")
    cpy = df.copy()
    scaler = MinMaxScaler()
    if method == 'standard':
        scaler = StandardScaler()
    cpy = pd.DataFrame(scaler.fit_transform(cpy), columns=cpy.columns)
    return cpy

> ### Split functions


In [15]:
def split_to_X_and_y(df, target_column='gender'):
    return df.drop([target_column], axis=1), df[target_column]
    
def split_to_train_and_test(X, y, test_ratio, rand_state):
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_ratio, random_state=rand_state)
    return X_train, X_test, y_train, y_test

> ### Full Preprocessing flow:

In [16]:
# Now we will run all the preprocessing functions on the data set.
# This function will return df scaled and vectorized.
def preprocess(df, method='tfidf', scale_method='standard', test=False):
    print ("Preprocessing...")
    cpy = df.copy()
    cpy = clear_df(cpy)
    if test:
        df = df.copy()
        X_test = clear_df(df)
        X_test_vec = vectorize(X_test, method, True)
        X_test_fselected = select_features(X_test_vec, np.zeros(X_test_vec.shape[0]))
        X_test_scaled = scale(X_test_fselected, scale_method)
        return X_test_scaled
    X, y = split_to_X_and_y(cpy)
    X_train, X_test, y_train, y_test = split_to_train_and_test(X, y, 0.2, 42)
    X_train_vec = vectorize(X_train, method)
    X_test_vec = vectorize(X_test, method, True)
    X_train_fslected = select_features(X_train_vec, y_train)
    X_test_fselected = select_features(X_test_vec, y_test)
    X_train_scaled = scale(X_train_fslected, scale_method)
    X_test_scaled = scale(X_test_fselected, scale_method)
    print ("Preprocessing done!")
    return X_train_scaled, X_test_scaled, X_train, X_test, y_train, y_test



In [17]:
# We will use the preprocess function on the train and test data sets.
X_train_scaled, X_test_scaled, X_train, X_test, y_train, y_test = preprocess(df_train)

Preprocessing...
Cleaning text...
Vectorizing...
Vectorizing...
Selecting features...
Selecting features...
Scaling...
Scaling...
Preprocessing done!


# Training:

> ### List of Models:

In [18]:
# Models we will use:
# 1. LinearSVC
# 2. Perceptron
# 3. GaussianNB
# 4. KNeighborsClassifier
# 5. LogisticRegression
models_and_params = [
    {
        'name': 'LinearSVC',
        'model': LinearSVC(),
        'params': {
            'C': [0.1, 1, 10, 100, 1000],
            'penalty': ['l1', 'l2'],
        }
    },
    {
        'name': 'Perceptron',
        'model': Perceptron(),
        'params': {
            'penalty': ['none', 'l2', 'l1', 'elasticnet'],
            'alpha': [0.0001, 0.05]
        }
    },
    {
        'name': 'GaussianNB',
        'model': GaussianNB(),
        'params': {
            'var_smoothing': [1e-09, 1e-08, 1e-07, 1e-06]
        }
    },
    {
        'name': 'KNeighborsClassifier',
        'model': KNeighborsClassifier(),
        'params': {
            'n_neighbors': [3, 5, 7, 9, 11],
            'weights': ['uniform', 'distance'],
            'algorithm': ['auto', 'ball_tree', 'kd_tree', 'brute'],
            'leaf_size': [10, 20, 30, 40, 50],
            'p': [1, 2]
        }
    },
    {
        'name': 'LogisticRegression',
        'model': LogisticRegression(),
        'params': {
            'penalty': ['l1', 'l2', 'elasticnet', 'none'],
            'C': [0.1, 1, 10, 100, 1000],
            'solver': ['newton-cg', 'lbfgs', 'liblinear', 'sag', 'saga'],
        }
    }                 
]

> ### Find the best parameters:

In [22]:
# We will use GridSearchCV to find the best parameters for each model.

def find_best_params(X, y, models_and_params):
    print ("Finding best params...")
    for model in models_and_params:
        grid = GridSearchCV(model['model'], model['params'], cv=5, n_jobs=-1)
        grid.fit(X, y)
        model['best_params'] = grid.best_params_
        model['best_score'] = grid.best_score_
    print ("Finding best params done!")

> ### Evaluate the models:

In [23]:
# We are going to evaluate the f1 score of each model using cross validation.

def evaluate_models(X, y, models_and_params):
    for model in models_and_params:
        print("Evaluating %s" % model['name'])
        model['model'].set_params(**model['best_params'])
        model['model'].fit(X, y)
        scores = cross_val_score(model['model'], X, y, cv=10, scoring=make_scorer(f1_score, average='macro'))
        y_pred = model['model'].predict(X_test_scaled)
        f1_m = f1_score(y_test, y_pred, pos_label='m')
        f1_f = f1_score(y_test, y_pred, pos_label='f')
        avg_f1 = np.mean(scores)
        confusion_mat = confusion_matrix(y_test, y_pred)
        print("Best params: \n%s" % (model['best_params']))
        print("F1 Male: %s" % (f1_m))
        print("F1 Female: %s" % (f1_f))
        print("Average f1 score: %s" % (avg_f1))
        print("Confusion matrix: \n%s" % (confusion_mat))
        print("--------------------------------------------------")
        model['scores'] = scores
        model['avg_f1'] = avg_f1
        model['confusion_mat'] = confusion_mat
        model['male_f1'] = f1_m
        model['female_f1'] = f1_f

In [24]:
# Now we will run the functions above on the data set.
find_best_params(X_train_scaled, y_train, models_and_params)
evaluate_models(X_train_scaled, y_train, models_and_params)


Finding best params...
Finding best params done!
Evaluating LinearSVC
Best params: 
{'C': 0.1, 'penalty': 'l2'}
F1 Male: 0.7443946188340809
F1 Female: 0.27848101265822783
Average f1 score: 0.7546933501299666
Confusion matrix: 
[[11 26]
 [31 83]]
--------------------------------------------------
Evaluating Perceptron
Best params: 
{'alpha': 0.0001, 'penalty': 'l2'}
F1 Male: 0.7918367346938776
F1 Female: 0.10526315789473685
Average f1 score: 0.8485566449236671
Confusion matrix: 
[[ 3 34]
 [17 97]]
--------------------------------------------------
Evaluating GaussianNB
Best params: 
{'var_smoothing': 1e-09}
F1 Male: 0.8603773584905661
F1 Female: 0.0
Average f1 score: 0.840068396589834
Confusion matrix: 
[[  0  37]
 [  0 114]]
--------------------------------------------------
Evaluating KNeighborsClassifier
Best params: 
{'algorithm': 'auto', 'leaf_size': 10, 'n_neighbors': 3, 'p': 1, 'weights': 'distance'}
F1 Male: 0.8603773584905661
F1 Female: 0.0
Average f1 score: 0.4747877172789005


> # Predicting the test data set

In [25]:
# We will use the best model to predict the test data set.
X_test_scaled = preprocess(df_test, test=True)
best_model = models_and_params[0] # LinearSVC is the best model that predicts male and woman.
y_pred = best_model['model'].predict(X_test_scaled)
df_predicted = pd.DataFrame({'test_example_id': df_test["test_example_id"], 'predicted_category': y_pred})
print ("Top 5 rows and bottom 5 rows of the predicted data set:")
df_predicted # print top 5 rows and bottom 5 rows


Preprocessing...
Cleaning text...
Cleaning text...
Vectorizing...
Selecting features...
Scaling...
Top 5 rows and bottom 5 rows of the predicted data set:


,test_example_id,predicted_category
0,0,m
1,1,f
2,2,f
3,3,m
4,4,f
...,...,...
318,318,m
319,319,m
320,320,f
321,321,m


In [26]:
# Check the distribution of the predicted categories.
df_predicted.value_counts(subset=['predicted_category'])

predicted_category
m                     247
f                      76
dtype: int64

### Save output to csv (optional)
After you're done save your output to the 'classification_results.csv' csv file.<br/>
We assume that the dataframe with your results contain the following columns:
* column 1 (left column): 'test_example_id'  - the same id associated to each of the test stories to be predicted.
* column 2 (right column): 'predicted_category' - the predicted gender value for each of the associated story. 

Assuming your predicted values are in the `df_predicted` dataframe, you should save you're results as following:

In [27]:
df_predicted.to_csv('classification_results.csv',index=False)